In [ ]:
!apt install unzip

Reading package lists... Done

Building dependency tree... Done

Reading state information... Done

unzip is already the newest version (6.0-26ubuntu3.2).

0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!unzip "/content/drive/MyDrive/Colab Notebooks/archive.zip" -d "content"

Streaming output truncated to the last 5000 lines.

  inflating: content/data/train/6/28451.png  

  inflating: content/data/train/6/28452.png  

  inflating: content/data/train/6/28467.png  

  inflating: content/data/train/6/28471.png  

  inflating: content/data/train/6/28472.png  

  inflating: content/data/train/6/28478.png  

  inflating: content/data/train/6/2848.png  

  inflating: content/data/train/6/28480.png  

  inflating: content/data/train/6/28484.png  

  inflating: content/data/train/6/28485.png  

  inflating: content/data/train/6/28493.png  

  inflating: content/data/train/6/28499.png  

  inflating: content/data/train/6/28507.png  

  inflating: content/data/train/6/28517.png  

  inflating: content/data/train/6/28519.png  

  inflating: content/data/train/6/28520.png  

  inflating: content/data/train/6/28524.png  

  inflating: content/data/train/6/28530.png  

  inflating: content/data/train/6/28531.png  

  inflating: content/data/train/6/28535.png  

  inflati

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from keras.callbacks import (EarlyStopping, ReduceLROnPlateau,
                                        TensorBoard)
from keras.optimizers import Adam
from keras.models import Model
import cv2
import pickle
from model import create_model
import matplotlib.pyplot as plt

In [11]:
def train():
    log_dir = './log'
    train_dataset_path=r"/content/content/data/train"
    val_dataset_path = r'/content/content/data/val'
    batch_size = 128

    lr = 1e-3
    epochs = 720
    num_classes=8
    train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range = 10,
        zoom_range = 0.1,
        horizontal_flip = True
       )

    train_generator = train_datagen.flow_from_directory(
        directory=train_dataset_path,
        target_size=(48, 48),
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode="categorical",
        shuffle=True,
        seed=42
    )
    test_datagen = ImageDataGenerator(
        rescale=1 / 255.0,)

    valid_generator = test_datagen.flow_from_directory(

        directory=val_dataset_path,
        target_size=(48, 48),
        color_mode="grayscale",
        class_mode="categorical",
        batch_size=batch_size,
        shuffle=True,
        seed=42
    )


    model = create_model(num_classes=num_classes)

    model.summary()
#     training_weights='./weights'
#     checkpoint_period = ModelCheckpoint(training_weights + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
#                                         monitor='val_loss', save_weights_only=True, save_best_only=False, period=1)
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
    tensorboard = TensorBoard(log_dir=log_dir)
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optimizer)
    model_info = model.fit(train_generator,validation_data=valid_generator,
                       epochs=epochs,callbacks=[tensorboard, early_stopping]
                       )

    model.evaluate(valid_generator,verbose=1)
    model.save('./model.h5')
    
if __name__ == '__main__':
    train()

Found 32645 images belonging to 8 classes.
Found 8166 images belonging to 8 classes.


Model: "fer_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 48, 48, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 48, 48, 256)    │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 48, 48, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 48, 48, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 24, 24, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 24, 24, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 24, 24, 384)    │     1,769,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 24, 24, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 12, 12, 384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 12, 12, 384)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 12, 12, 192)    │       663,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 12, 12, 192)    │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 6, 6, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 6, 6, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 6, 6, 384)      │       663,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 6, 6, 384)      │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 3, 3, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_28 (Dropout)            │ (None, 3, 3, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 3456)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_29 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,174,216 (19.74 MB)

 Trainable params: 5,170,760 (19.72 MB)

...

W0000 00:00:1716283031.866128     115 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
2024-05-21 09:17:47.268944: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng36{k2=3,k3=0} for conv (f32[102,512,48,48]{3,2,1,0}, u8[0]{0}) custom-call(f32[102,256,48,48]{3,2,1,0}, f32[512,256,3,3]{3,2,1,0}, f32[512]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-05-21 09:17:47.368200: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.099383575s
Trying algorithm eng36{k2=3,k3=0} for conv (f32[102,512,48,48]{3,2,1,0}, u8[0]{0}) custom-call(f32[102,256,48,48]{3,2,1,0}, f32[512,256,3,3]{3,2,1,0}, f32[512]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convB

256/256 ━━━━━━━━━━━━━━━━━━━━ 282s 685ms/step - accuracy: 0.2043 - loss: 2.3748 - val_accuracy: 0.2207 - val_loss: 2.3062
Epoch 2/720


W0000 00:00:1716283084.800616     114 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


256/256 ━━━━━━━━━━━━━━━━━━━━ 85s 328ms/step - accuracy: 0.3321 - loss: 1.7449 - val_accuracy: 0.3097 - val_loss: 1.9733
Epoch 3/720
256/256 ━━━━━━━━━━━━━━━━━━━━ 85s 327ms/step - accuracy: 0.4091 - loss: 1.5429 - val_accuracy: 0.4276 - val_loss: 1.4898
Epoch 4/720
256/256 ━━━━━━━━━━━━━━━━━━━━ 84s 324ms/step - accuracy: 0.4702 - loss: 1.3676 - val_accuracy: 0.2551 - val_loss: 1.9248
Epoch 5/720
256/256 ━━━━━━━━━━━━━━━━━━━━ 84s 322ms/step - accuracy: 0.4627 - loss: 1.4023 - val_accuracy: 0.5097 - val_loss: 1.2721
Epoch 6/720
256/256 ━━━━━━━━━━━━━━━━━━━━ 142s 320ms/step - accuracy: 0.5236 - loss: 1.2488 - val_accuracy: 0.4369 - val_loss: 1.4842
Epoch 7/720
256/256 ━━━━━━━━━━━━━━━━━━━━ 83s 318ms/step - accuracy: 0.5385 - loss: 1.2144 - val_accuracy: 0.5482 - val_loss: 1.2243
Epoch 8/720
256/256 ━━━━━━━━━━━━━━━━━━━━ 86s 332ms/step - accuracy: 0.5356 - loss: 1.2063 - val_accuracy: 0.5685 - val_loss: 1.1486
Epoch 9/720
256/256 ━━━━━━━━━━━━━━━━━━━━ 83s 319ms/step - accuracy: 0.5590 - loss: 1.15

KeyError: 'acc'